In [1]:
!pip install ptflops
!pip install pytorch_msssim

In [2]:
from torch.utils.data import Dataset
from torchvision.transforms import transforms
from PIL import Image
class CustomDataset(Dataset):
    def __init__(self, files, transform=None):
        self.transform = transform
        self.files = files
    def __len__(self):
        return len(self.files)
    def __getitem__(self, idx):
        in_image = Image.open("/kaggle/input/files-for-training/Dataset/SIH_24/Noisy Images/"+self.files[idx])
        transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor()])
        out_image = Image.open("/kaggle/input/files-for-training/Dataset/SIH_24/Ground Truth/"+self.files[idx]).convert('L')
        return transform(in_image), transform(out_image)

In [ ]:
import sys
sys.path.insert(1, "/kaggle/input/rdunet")
import yaml
import torch
import torch.optim as optim
from os.path import join
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from ptflops import get_model_complexity_info

from model import RDUNet
from data_management import NoisyImagesDataset, DataSampler
from train import fit_model
from transforms import AdditiveWhiteGaussianNoise, RandomHorizontalFlip, RandomVerticalFlip, RandomRot90
from utils import set_seed


def main():
    with open('/kaggle/input/rdunet/config.yaml', 'r') as stream:            # Load YAML configuration file.
        config = yaml.safe_load(stream)

    model_params = config['model']
    model_params["channels"] = 1
    train_params = config['train']
    val_params = config['val']

    # Defining model:
    set_seed(0)
    model = RDUNet(**model_params)

    print('Model summary:')
    test_shape = (model_params['channels'], train_params['patch size'], train_params['patch size'])
    with torch.no_grad():
        macs, params = get_model_complexity_info(model, test_shape, as_strings=True,
                                                 print_per_layer_stat=False, verbose=False)
        print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
        print('{:<30}  {:<8}'.format('Number of parameters: ', params))

    # Define the model name and use multi-GPU if it is allowed.
    model_name = 'model_color' if model_params['channels'] == 3 else 'model_gray'
    device = torch.device(train_params['device'])
    print("Using device: {}".format(device))
    if torch.cuda.device_count() > 1 and 'cuda' in device.type and train_params['multi gpu']:
        model = nn.DataParallel(model)
        print('Using multiple GPUs')

    model = model.to(device)
    param_group = []
    for name, param in model.named_parameters():
        if 'conv' in name and 'weight' in name:
            p = {'params': param, 'weight_decay': train_params['weight decay']}
        else:
            p = {'params': param, 'weight_decay': 0.}
        param_group.append(p)

    # Load training and validation file names.
    # Modify .txt files if datasets do not fit in memory.
    with open('/kaggle/input/files-for-training/train_files.txt', 'r') as f_train, open('/kaggle/input/files-for-training/valid_files.txt', 'r') as f_val:
        raw_train_files = f_train.read().splitlines()
        raw_val_files = f_val.read().splitlines()
        #train_files = list(map(lambda file: join("/kaggle/input/files-for-training/Dataset/SIH_24/Noisy Images", file), raw_train_files))
        #val_files = list(map(lambda file: join("/kaggle/input/files-for-training/Dataset/SIH_24/Noisy Images", file), raw_val_files))

    training_transforms = transforms.Compose([
        RandomHorizontalFlip(),
        RandomVerticalFlip(),
        RandomRot90()
    ])

    # Predefined noise level
    #train_noise_transform = [AdditiveWhiteGaussianNoise(train_params['noise level'], clip=True)]
    #val_noise_transforms = [AdditiveWhiteGaussianNoise(s, fix_sigma=True, clip=True) for s in val_params['noise levels']]
    
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()])
    print('\nLoading training dataset:')
    training_dataset = CustomDataset(raw_train_files, transform)

    print('\nLoading validation dataset:')
    validation_dataset = CustomDataset(raw_val_files)
    
    # Training in sub-epochs:
    print('Training patches:', len(training_dataset))
    print('Validation patches:', len(validation_dataset))
    n_samples = len(training_dataset) // train_params['dataset splits']
    n_epochs = 10
    sampler = DataSampler(training_dataset, num_samples=n_samples)

    data_loaders = {
        'train': DataLoader(training_dataset, 1, num_workers=train_params['workers']),
        'val': DataLoader(validation_dataset, 1, num_workers=val_params['workers']),
    }

    # Optimization:
    learning_rate = train_params['learning rate']
    step_size = train_params['scheduler step'] * train_params['dataset splits']

    criterion = nn.L1Loss()
    optimizer = optim.AdamW(param_group, lr=learning_rate)
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=train_params['scheduler gamma'])

    # Train the model
    fit_model(model, data_loaders, model_params['channels'], criterion, optimizer, lr_scheduler, device,
              n_epochs, val_params['frequency'], train_params['checkpoint path'], model_name)
    return model
model = main()

Model summary:
Computational complexity:       50.48 GMac
Number of parameters:           166.37 M
Using device: cuda:0

Loading training dataset:

Loading validation dataset:
Training patches: 149
Validation patches: 50

Epoch: 1/10 - Learning rate: 1.0000e-04


/opt/conda/lib/python3.10/site-packages/torch/utils/data/sampler.py:65: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn("`data_source` argument is not used and will be removed in 2.2.0."
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what 


Epoch: 2/10 - Learning rate: 1.0000e-04


Training - Loss:2.22081e-01 - PSNR:25.98713 - SSIM:0.53900: 100%|███████████| 149/149 [02:05<00:00,  1.19it/s]
Validation - Loss:1.45881e+02 - PSNR:-45.63456 - SSIM:0.00000: 100%|██████████| 50/50 [00:14<00:00,  3.43it/s]


Epoch: 3/10 - Learning rate: 1.0000e-04


...


Training - Loss:3.81399e-02 - PSNR:26.33822 - SSIM:0.49924: 100%|███████████| 149/149 [02:04<00:00,  1.19it/s]
Validation - Loss:3.74628e-02 - PSNR:26.45665 - SSIM:0.51506: 100%|███████████| 50/50 [00:14<00:00,  3.44it/s]


Epoch: 9/10 - Learning rate: 1.0000e-04



Training - Loss:3.73939e-02 - PSNR:26.48174 - SSIM:0.50606: 100%|███████████| 149/149 [02:05<00:00,  1.19it/s]
Validation - Loss:3.65013e-02 - PSNR:26.63057 - SSIM:0.52578: 100%|███████████| 50/50 [00:14<00:00,  3.44it/s]


Epoch: 10/10 - Learning rate: 1.0000e-04



Training - Loss:3.62483e-02 - PSNR:26.73592 - SSIM:0.51206:  44%|█████▎      | 66/149 [00:55<01:09,  1.19it/s]

/kaggle/working/model.zip

NameError: name 'model' is not defined